GDP	26.0	
Mother's qualification	
Application mode	
Application order	
Father's qualification	
Curricular units 2nd sem (credited)	
Inflation rate	
Curricular units 2nd sem (without evaluations)	
Previous qualification	
Curricular units 1st sem (without evaluations)	
Displaced	
Marital status	
Daytime/evening attendance	
Nacionality	
Educational special needs	
International	


In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

C:\Users\ADM\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = "Dropout_Academic Success - Sheet1.csv"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "ankanhore545/dropout-or-academic-success",
  file_path,
)

C:\Users\ADM\AppData\Local\Temp\ipykernel_8120\2961937467.py:3: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


In [4]:
df_novo = df.drop(columns=["GDP", "Mother's qualification", "Application mode", "Application order", "Father's qualification", "Curricular units 2nd sem (credited)", "Inflation rate", "Curricular units 2nd sem (without evaluations)", "Previous qualification", "Curricular units 1st sem (without evaluations)", "Displaced", "Marital status", "Daytime/evening attendance", "Nacionality", "Educational special needs", "International"])


In [5]:
df_novo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 21 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Course                                  4424 non-null   int64  
 1   Previous qualification (grade)          4424 non-null   float64
 2   Mother's occupation                     4424 non-null   int64  
 3   Father's occupation                     4424 non-null   int64  
 4   Admission grade                         4424 non-null   float64
 5   Debtor                                  4424 non-null   int64  
 6   Tuition fees up to date                 4424 non-null   int64  
 7   Gender                                  4424 non-null   int64  
 8   Scholarship holder                      4424 non-null   int64  
 9   Age at enrollment                       4424 non-null   int64  
 10  Curricular units 1st sem (credited)     4424 non-null   int6

In [6]:
X = df_novo.drop(columns=['Target'])
y = df_novo['Target']

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y) # Convertendo rótulos para valores numéricos

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X) # Padronizando os recursos

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.2,
    random_state=42,
    stratify=y # Mantendo a proporção das classes
)

In [9]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)

class_weight_dict = dict(zip(classes, class_weights))
print(class_weight_dict)


{np.int64(0): np.float64(1.0375256523013778), np.int64(1): np.float64(1.857742782152231), np.int64(2): np.float64(0.6676098849273722)}


In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

models = {
    "Naive Bayes": GaussianNB(),

    "SVM (Linear)": LinearSVC(
        class_weight=class_weight_dict,
        random_state=42
    ),

    "Logistic Regression": LogisticRegression(
        max_iter=1000,
        class_weight=class_weight_dict
    ),

    "Decision Tree": DecisionTreeClassifier(
        class_weight=class_weight_dict,
        random_state=42
    ),

    "Random Forest": RandomForestClassifier(
        n_estimators=200,
        class_weight=class_weight_dict,
        random_state=42
    ),

    "XGBoost": XGBClassifier(
        objective="multi:softmax",
        num_class=len(classes),
        eval_metric="mlogloss",
        random_state=42
    ),

    "CatBoost": CatBoostClassifier(
        loss_function="MultiClass",
        class_weights=list(class_weight_dict.values()),
        verbose=0,
        random_state=42
    )
}


In [11]:
from sklearn.model_selection import StratifiedKFold, cross_validate

cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

scoring = {
    "accuracy": "accuracy",
    "precision": "precision_macro",
    "recall": "recall_macro",
    "f1": "f1_macro"
}

In [12]:
results = []

for name, model in models.items():
    scores = cross_validate(
        model,
        X_train,
        y_train,
        cv=cv,
        scoring=scoring,
        n_jobs=-1
    )

    results.append({
        "Model": name,
        "Accuracy": scores["test_accuracy"].mean(),
        "Precision": scores["test_precision"].mean(),
        "Recall": scores["test_recall"].mean(),
        "F1": scores["test_f1"].mean()
    })

In [13]:
import pandas as pd
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="Recall", ascending=False)

results_df

,Model,Accuracy,Precision,Recall,F1
2,Logistic Regression,0.744838,0.711459,0.709180,0.702000
6,CatBoost,0.762641,0.710154,0.702990,0.705026
5,XGBoost,0.767446,0.714828,0.692878,0.699721
4,Random Forest,0.771967,0.718335,0.679031,0.688073
1,SVM (Linear),0.755291,0.702901,0.678042,0.686360
3,Decision Tree,0.681270,0.616516,0.615377,0.615466
0,Naive Bayes,0.697646,0.618624,0.588333,0.590360
